In [7]:
# load the packages needed
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from src.solver import Solver
from src.model import AR, MA
from src.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
import src.data_processor as dp
import src.inference 


In [8]:
from src.ts_gen import ar1_gen
data = np.loadtxt("../data/SP500array.csv", delimiter=',')
X=np.array([data[0:100,0]])
# Y=dp.get_return(X)
Y = ar1_gen(0.5, sigma = 1.0, time = 200, num = 1, burnin = 2000)

In [9]:
lag = 1
sigma = 2.0
intercept = 0.1
phi = np.random.randn(1, 1)
AR_model = AR(lag=lag, phi=phi, sigma=sigma, intercept=intercept)
AR_model.params

{'intercept': 0.1, 'phi': array([[-0.42254133]]), 'sigma': 2.0}

In [10]:
_, grads = AR_model.loss(Y)


# define a useful mathod for future use
def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))


d_phi = grads['phi']
d_intercept = grads['intercept']

d_sigma = grads['sigma']

d_phi_num = eval_numerical_gradient_array(lambda phi: AR_model.loss(Y, lag,phi,sigma,intercept)[0], phi, 1)
print 'd_phi error: ', rel_error(d_phi_num, d_phi)

d_intercept_num = eval_numerical_gradient_array(lambda intercept: AR_model.loss(Y, lag,phi,sigma,intercept)[0], intercept, 1)
print 'd_intercept error: ', rel_error(d_intercept_num, d_intercept)

d_sigma_num = eval_numerical_gradient_array(lambda sigma: AR_model.loss(Y, lag,phi,sigma,intercept)[0], sigma, 1)
print 'd_sigma error: ', rel_error(d_sigma_num, d_sigma)

d_phi error:  3.66232459651e-12
d_intercept error:  1.68217619673e-11
d_sigma error:  1.93000722028e-11


In [ ]:

solver = Solver(AR_model, Y,
                          update_rule='sgd',
                          optim_config={
                            'learning_rate': 1e-5,
                          },
                          
                          num_epochs=3000, batch_size=1,
                          print_every=100)
solver.train()
AR_model.params

3000


In [6]:
solver = Solver(AR_model, Y,
                          update_rule='sgd_momentum',
                          optim_config={
                            'learning_rate': 1e-6,
                          },
                          
                          num_epochs=10000, batch_size=1,
                          print_every=100)
solver.train()
AR_model.params

10000
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098
the loss is 295.655098


KeyboardInterrupt: 

In [16]:
AR_model.predict(X,10)

array([[ 0.01173585,  0.03953278,  0.05445403,  0.06246367,  0.06676321,
         0.06907117,  0.07031008,  0.07097512,  0.0713321 ,  0.07152373]])

In [18]:
from src.ts_gen import ar1_gen
import src.inference as inf

x = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)
X=np.array([data[0:100,0]])
Y=dp.get_return(X)

y = inf.yule_walker(x, order =1, method = "mle")
phi,sigma = inf.yule_walker(Y, order =2, method = "mle")
print phi 
print sigma



[-0.19786273 -0.05678889]
0.000119785804031


In [19]:

phi

array([-0.19786273, -0.05678889])

In [20]:
AR_model = AR(lag=2, phi=phi, sigma=sigma, intercept=0)

In [24]:
rt=AR_model.predict(Y,10)

In [25]:
dp.get_price(X[99,0],rt)

IndexError: index 99 is out of bounds for axis 0 with size 1